## This script will extract data off avito for further analysis

In [195]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from IPython.display import clear_output
import numpy as np

In [196]:
#this functions takes an avito car page and extracts data from it
def getInternalData(link):
    data={}
    page=requests.get(link)
    print(link)
    soup = BeautifulSoup(page.content,"html.parser")
    annonceText=soup.find("p",{"style":"word-wrap: break-word;"}).text
    dataPanel=soup.find("aside",{"class":"panel panel-body panel-info"})
    dataList=dataPanel.find("ul")
    for li in dataList.find_all("li"):
        if li.find("strong")!=None:
            title=li.find("strong").text
            
            if li.find("a")==None:
                
                value=li.find("h2").text.split("\n")[1]
            else:
                value=li.find("a").text
            data[title]=value
        else:
            pass
       
    return data
    
    

In [197]:
#setup how many pages you want using pageDeepness
pageDeepness=300



totalItems=pageDeepness*35 
currentPage=1
data=[]
for i in range(1,pageDeepness+1):
    clear_output()
    print("--------------Currently processing page "+str(i)+" -------------------------------------------------------")
    
    
    startItem=(i-1)*35 +1
    endItem=startItem+35
    page=requests.get("https://www.avito.ma/fr/maroc/voitures-à_vendre?o="+str(i))
    soup = BeautifulSoup(page.content,"html.parser")
    for j in range(startItem,endItem):
        idItem="li-item-"+str(j)+" "
        item=soup.find("div", {"id": idItem})
        name=item.find("h2").find("a").text
        price=item.find("span",{"class":"price_value"}).text
        city=item.find("span",{"class":"item-info-extra fs14"}).find("a").text
        proOrNot=False if item.find("span",{"class":"item-info-extra fs14"}).find("strong")==None else True
        link=item.find("h2").find("a",href=True)['href']
        try :
            dataPoint=getInternalData(link)
        except:
            continue
        dataPoint["name"]=name
        dataPoint["city"]=city
        dataPoint["proOrNot"]=proOrNot
        dataPoint["price"]=price
        data.append(dataPoint)
        print("progress: "+str(j/totalItems)+"%")

clear_output()       
print("All Done :3")
        
    


    

    
    

--------------Currently processing page 268 -------------------------------------------------------
https://www.avito.ma/fr/sidi_moumen/voitures/Megan_bon_etat_a_vendre_36290154.htm
progress: 0.890095238095238%
https://www.avito.ma/fr/salé/voitures/Fiat_punto_essence__36290131.htm
progress: 0.8901904761904762%
https://www.avito.ma/fr/hay_riad/voitures/Passat_cc_Cara_36290110.htm
progress: 0.8902857142857142%
https://www.avito.ma/fr/tanger/voitures/golf_3_36290109.htm
progress: 0.8903809523809524%


AttributeError: 'NoneType' object has no attribute 'text'

In [198]:
#put it into a pandas df
dataframe=pd.DataFrame(data)
dataframe.shape

(9336, 12)

## Export it to excel

In [199]:
dataframe.to_excel("avito_data.xlsx")

## TO DO :
* Clean the data
* Make a model to predict price


## Cleaning data

In [223]:
#load data
df=pd.read_excel('avito_data.xlsx')

In [224]:
df.head()

,Unnamed: 0,Année-Modèle:,Kilométrage:,Marque:,Modèle:,Puissance fiscale:,Secteur:,Type de carburant:,Type:,city,name,price,proOrNot
0,0,2006,180 000 - 189 999,Kia,Sorento,9 CV,ain_sebaa,Diesel,"Voitures, Offre",Casablanca,Kia Sorento Diesel -2006,75 000,True
1,1,2018,35 000 - 39 999,Mercedes-Benz,Classe E,8 CV,derb_ghallef,Diesel,"Voitures, Offre",Casablanca,Mercedes-Benz Classe E220 COUPE -2018,,True
2,2,2015,140 000 - 149 999,Mercedes-Benz,Classe E,8 CV,NaN,Diesel,"Voitures, Offre",Fès,Mercedes Classe E -2015,300 000,False
3,3,2002,250 000 - 299 999,Fiat,Uno,5 CV,NaN,Essence,"Voitures, Offre",Agadir,fiat uno -2002,28 000,False
4,4,2008,100 000 - 109 999,BMW,X5,18,bourgogne,Essence,"Voitures, Offre",Casablanca,BMW X5 Essence -2008,129 000,True


In [225]:
#clean 
df.columns

Index(['Unnamed: 0', 'Année-Modèle:', 'Kilométrage:', 'Marque:', 'Modèle:',
       'Puissance fiscale:', 'Secteur:', 'Type de carburant:', 'Type:', 'city',
       'name', 'price', 'proOrNot'],
      dtype='object')

In [226]:
#drop unamed col
df.drop("Unnamed: 0",inplace=True,axis=1)

In [227]:
#drop type col 
df.drop("Type:",inplace=True,axis=1)


In [228]:
#Turn kilometrage into kilo min and kilo max
df["kiloMin"]=df["Kilométrage:"].apply(lambda s : s.split("-")[0].strip().replace(" ",""))
df["kiloMax"]=df["Kilométrage:"].apply(lambda s : s.split("-")[1].strip().replace(" ","") if len(s.split("-"))==2 else s.split("-")[0].strip().replace(" ",""))
df.drop("Kilométrage:",inplace=True,axis=1)

In [229]:
#drop name
df.drop("name",inplace=True,axis=1)

In [230]:
df.head()

,Année-Modèle:,Marque:,Modèle:,Puissance fiscale:,Secteur:,Type de carburant:,city,price,proOrNot,kiloMin,kiloMax
0,2006,Kia,Sorento,9 CV,ain_sebaa,Diesel,Casablanca,75 000,True,180000,189999
1,2018,Mercedes-Benz,Classe E,8 CV,derb_ghallef,Diesel,Casablanca,,True,35000,39999
2,2015,Mercedes-Benz,Classe E,8 CV,NaN,Diesel,Fès,300 000,False,140000,149999
3,2002,Fiat,Uno,5 CV,NaN,Essence,Agadir,28 000,False,250000,299999
4,2008,BMW,X5,18,bourgogne,Essence,Casablanca,129 000,True,100000,109999


In [233]:
#turn puiss fiscal into number
df['Puissance fiscale:']=df['Puissance fiscale:'].apply(lambda s : s.split(" ")[0].strip() if type(s)!=float else s)


In [234]:
df.head()

,Année-Modèle:,Marque:,Modèle:,Puissance fiscale:,Secteur:,Type de carburant:,city,price,proOrNot,kiloMin,kiloMax
0,2006,Kia,Sorento,9,ain_sebaa,Diesel,Casablanca,75 000,True,180000,189999
1,2018,Mercedes-Benz,Classe E,8,derb_ghallef,Diesel,Casablanca,,True,35000,39999
2,2015,Mercedes-Benz,Classe E,8,NaN,Diesel,Fès,300 000,False,140000,149999
3,2002,Fiat,Uno,5,NaN,Essence,Agadir,28 000,False,250000,299999
4,2008,BMW,X5,18,bourgogne,Essence,Casablanca,129 000,True,100000,109999


In [235]:
#make columns names prettier
df.columns=['annéeModèle',"Marque","Modèle","puissFisacale","Secteur","typeCarburant","ville","prix","pro","kiloMin","kiloMax"]
df.head()


,annéeModèle,Marque,Modèle,puissFisacale,Secteur,typeCarburant,ville,prix,pro,kiloMin,kiloMax
0,2006,Kia,Sorento,9,ain_sebaa,Diesel,Casablanca,75 000,True,180000,189999
1,2018,Mercedes-Benz,Classe E,8,derb_ghallef,Diesel,Casablanca,,True,35000,39999
2,2015,Mercedes-Benz,Classe E,8,NaN,Diesel,Fès,300 000,False,140000,149999
3,2002,Fiat,Uno,5,NaN,Essence,Agadir,28 000,False,250000,299999
4,2008,BMW,X5,18,bourgogne,Essence,Casablanca,129 000,True,100000,109999


In [236]:
df.dtypes

annéeModèle      object
Marque           object
Modèle           object
puissFisacale    object
Secteur          object
typeCarburant    object
ville            object
prix             object
pro                bool
kiloMin          object
kiloMax          object
dtype: object

In [237]:
#fix null values
#prix
df["prix"]=df["prix"].apply(lambda s : np.NaN if s=="\xa0" else s.replace(" ","") )
df["kiloMax"]=df["kiloMax"].apply(lambda s : np.NaN if s=='' else s.replace(" ","") )
df["kiloMin"]=df["kiloMin"].apply(lambda s : np.NaN if s=='' else s.replace(" ","") )


In [238]:
#puissFiscale
df["puissFisacale"]=df["puissFisacale"].apply(lambda s : np.NaN if s=="-" else s )
df["Marque"]=df["Marque"].apply(lambda s : np.NaN if s=="-" else s )
df["Modèle"]=df["Modèle"].apply(lambda s : np.NaN if s=="-" else s )
df["annéeModèle"]=df["annéeModèle"].apply(lambda s : np.NaN if s=="-" else s )
df["typeCarburant"]=df["typeCarburant"].apply(lambda s : np.NaN if s=="-" else s )

In [239]:
df["kiloMin"][2]

'140000'

In [241]:
#fix numerical values

df["kiloMin"]=pd.to_numeric(df["kiloMin"], errors='coerce')
df["kiloMax"]=pd.to_numeric(df["kiloMax"], errors='coerce')
df["prix"]=pd.to_numeric(df["prix"], errors='coerce')
df["puissFisacale"]=pd.to_numeric(df["puissFisacale"], errors='coerce')

In [242]:
df.dtypes

annéeModèle       object
Marque            object
Modèle            object
puissFisacale    float64
Secteur           object
typeCarburant     object
ville             object
prix             float64
pro                 bool
kiloMin          float64
kiloMax          float64
dtype: object

In [253]:
#fix categorical values
df["typeCarburant"]=df["typeCarburant"].astype("category")
df["Marque"]=df["Marque"].astype("category")
df["Modèle"]=df["Modèle"].astype("category")
df["Secteur"]=df["Secteur"].astype("category")
df["ville"]=df["ville"].astype("category")
df["annéeModèle"]=df["annéeModèle"].astype("category")

df.dtypes

annéeModèle      category
Marque           category
Modèle           category
puissFisacale     float64
Secteur          category
typeCarburant    category
ville            category
prix              float64
pro                  bool
kiloMin           float64
kiloMax           float64
dtype: object

In [254]:
df.head()

,annéeModèle,Marque,Modèle,puissFisacale,Secteur,typeCarburant,ville,prix,pro,kiloMin,kiloMax
0,2006,Kia,Sorento,9.0,ain_sebaa,Diesel,Casablanca,75000.0,True,180000.0,189999.0
1,2018,Mercedes-Benz,Classe E,8.0,derb_ghallef,Diesel,Casablanca,NaN,True,35000.0,39999.0
2,2015,Mercedes-Benz,Classe E,8.0,NaN,Diesel,Fès,300000.0,False,140000.0,149999.0
3,2002,Fiat,Uno,5.0,NaN,Essence,Agadir,28000.0,False,250000.0,299999.0
4,2008,BMW,X5,18.0,bourgogne,Essence,Casablanca,129000.0,True,100000.0,109999.0


In [255]:
df["prix"].isnull().sum()

2310